In [1]:
import torch

from torch.utils.data import DataLoader
from torcheval.metrics import R2Score, BinaryAccuracy, BinaryAUROC

from molsetrep.utils.torch_trainer import TorchTrainer
from molsetrep.utils.datasets import molnet_loader
from molsetrep.utils.converters import molnet_to_pyg
from molsetrep.utils.root_mean_squared_error import RootMeanSquaredError
from molsetrep.utils.imbalanced_sampler import ImbalancedSampler
from molsetrep.models import SetRepClassifier, SetRepRegressor
from molsetrep.encoders import SECMQNFPEncoder

Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'pytorch_lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'


## Classification

In [13]:
train, valid, test = molnet_loader("bace_classification")
enc = SECMQNFPEncoder()

train_dataset = enc.encode(train.ids, [y[0] for y in train.y], label_dtype=torch.long)
valid_dataset = enc.encode(valid.ids, [y[0] for y in valid.y], label_dtype=torch.long)
test_dataset = enc.encode(test.ids, [y[0] for y in test.y], label_dtype=torch.long)

train_loader = DataLoader(train_dataset, batch_size=64, sampler=ImbalancedSampler(train_dataset))
valid_loader = DataLoader(valid_dataset, batch_size=64)
test_loader = DataLoader(test_dataset, batch_size=64)

In [15]:
d = 44
model = SetRepClassifier(100, 16, d, 2)

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
criterion = torch.nn.NLLLoss()

trainer = TorchTrainer(
    model,
    optimizer,
    criterion,
    200,
    [BinaryAccuracy(), BinaryAUROC()],
    [BinaryAccuracy(), BinaryAUROC()],
    [BinaryAccuracy(), BinaryAUROC()],
    # scheduler=scheduler,
    monitor_metric=1,
    monitor_lower_is_better=False
)

trainer.train(train_loader, valid_loader)
trainer.test(test_loader)

*  Epoch 1: Train loss: 0.679 (BinaryAccuracy: 0.57, BinaryAUROC: 0.563)  Valid loss: 0.659 (BinaryAccuracy: 0.563, BinaryAUROC: 0.523)
|  Epoch 2: Train loss: 0.681 (BinaryAccuracy: 0.576, BinaryAUROC: 0.572)  Valid loss: 0.666 (BinaryAccuracy: 0.563, BinaryAUROC: 0.523)
|  Epoch 3: Train loss: 0.672 (BinaryAccuracy: 0.609, BinaryAUROC: 0.603)  Valid loss: 0.658 (BinaryAccuracy: 0.563, BinaryAUROC: 0.521)
|  Epoch 4: Train loss: 0.668 (BinaryAccuracy: 0.616, BinaryAUROC: 0.617)  Valid loss: 0.648 (BinaryAccuracy: 0.556, BinaryAUROC: 0.514)
*  Epoch 5: Train loss: 0.655 (BinaryAccuracy: 0.668, BinaryAUROC: 0.667)  Valid loss: 0.637 (BinaryAccuracy: 0.589, BinaryAUROC: 0.546)
|  Epoch 6: Train loss: 0.65 (BinaryAccuracy: 0.675, BinaryAUROC: 0.678)  Valid loss: 0.63 (BinaryAccuracy: 0.589, BinaryAUROC: 0.546)
|  Epoch 7: Train loss: 0.641 (BinaryAccuracy: 0.702, BinaryAUROC: 0.702)  Valid loss: 0.63 (BinaryAccuracy: 0.563, BinaryAUROC: 0.529)
*  Epoch 8: Train loss: 0.639 (BinaryAccuracy

## Regression

In [13]:
train, valid, test = molnet_loader("lipo")
train_loader, valid_loader, test_loader = molnet_to_pyg(train, valid, test, label_type=torch.float)

num_node_features = train_loader.dataset[0].num_node_features
num_edge_features = train_loader.dataset[0].num_edge_features
model = GNNSetRepRegressor(num_node_features, 256, 2, num_edge_features, 100, 16)

optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)
criterion = torch.nn.MSELoss()

trainer = Trainer(
    model,
    optimizer,
    criterion,
    200,
    [R2Score(), RootMeanSquaredError()],
    [R2Score(), RootMeanSquaredError()],
    [R2Score(), RootMeanSquaredError()],
    monitor_metric=1
)

trainer.train(train_loader, valid_loader)
trainer.test(test_loader)

/home/daenu/miniconda3/envs/molsetrep/lib/python3.9/site-packages/deepchem/feat/molecule_featurizers/coulomb_matrices.py:145: RuntimeWarning: divide by zero encountered in divide
  m = np.outer(z, z) / d


KeyError: 'bond_type'